In [39]:
!pip install gluonnlp pandas tqdm
!pip install mxnet
!pip install sentencepiece==0.1.91
!pip install transformers==4.8.2
!pip install torch

In [40]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-na5sguk5/kobert-tokenizer_bda229cbd16f4e6f84c86aed28db8b59
  Running command git clone -q https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-na5sguk5/kobert-tokenizer_bda229cbd16f4e6f84c86aed28db8b59


In [41]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master 

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-oo5ab92b
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-oo5ab92b


In [42]:
!pip uninstall folium

Found existing installation: folium 0.2.1
Uninstalling folium-0.2.1:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/folium-0.2.1.dist-info/*
    /usr/local/lib/python3.7/dist-packages/folium/*
Proceed (y/n)? y
  Successfully uninstalled folium-0.2.1


In [43]:
!pip install folium==0.2.1

  Using cached folium-0.2.1-py3-none-any.whl


In [44]:
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [45]:
from kobert_tokenizer import KoBERTTokenizer

In [46]:
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel, vocab = get_pytorch_kobert_model()
# print(bertModel, vocab_obj)

using cached model. /content/.cache/kobert_v1.zip
using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [47]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
import pandas as pd

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
from transformers import BertModel

#GPU 사용 시
device = torch.device("cuda:0")

In [48]:
import os

n_devices = torch.cuda.device_count()
print(n_devices)

for i in range(n_devices):
    print(torch.cuda.get_device_name(i))

1
Tesla P100-PCIE-16GB


In [49]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [50]:
import pandas as pd
data = pd.read_csv('/content/drive/MyDrive/new_data_model1/한국어_단발성_대화_데이터셋.csv')
data

,Sentence,Emotion,Unnamed: 2,Unnamed: 3,Unnamed: 4,공포,5468
0,언니 동생으로 부르는게 맞는 일인가요..??,공포,NaN,NaN,NaN,놀람,5898.0
1,그냥 내 느낌일뿐겠지?,공포,NaN,NaN,NaN,분노,5665.0
2,아직너무초기라서 그런거죠?,공포,NaN,NaN,NaN,슬픔,5267.0
3,유치원버스 사고 낫다던데,공포,NaN,NaN,NaN,중립,4830.0
4,근데 원래이런거맞나요,공포,NaN,NaN,NaN,행복,6037.0
...,...,...,...,...,...,...,...
38589,솔직히 예보 제대로 못하는 데 세금이라도 아끼게 그냥 폐지해라..,혐오,NaN,NaN,NaN,NaN,NaN
38590,재미가 없으니 망하지,혐오,NaN,NaN,NaN,NaN,NaN
38591,공장 도시락 비우생적임 아르바이트했는데 화장실가성 손도 않씯고 재료 담고 바닥 떨어...,혐오,NaN,NaN,NaN,NaN,NaN
38592,코딱지 만한 나라에서 지들끼리 피터지게 싸우는 센징 클래스 ㅉㅉㅉ,혐오,NaN,NaN,NaN,NaN,NaN


In [51]:
data = data[['Sentence', 'Emotion']]
data

,Sentence,Emotion
0,언니 동생으로 부르는게 맞는 일인가요..??,공포
1,그냥 내 느낌일뿐겠지?,공포
2,아직너무초기라서 그런거죠?,공포
3,유치원버스 사고 낫다던데,공포
4,근데 원래이런거맞나요,공포
...,...,...
38589,솔직히 예보 제대로 못하는 데 세금이라도 아끼게 그냥 폐지해라..,혐오
38590,재미가 없으니 망하지,혐오
38591,공장 도시락 비우생적임 아르바이트했는데 화장실가성 손도 않씯고 재료 담고 바닥 떨어...,혐오
38592,코딱지 만한 나라에서 지들끼리 피터지게 싸우는 센징 클래스 ㅉㅉㅉ,혐오


In [52]:
data.isnull().sum()

Sentence    0
Emotion     0
dtype: int64

In [53]:
data.loc[(data['Emotion']=='공포'), 'Emotion'] = 0
data.loc[(data['Emotion']=='놀람'), 'Emotion'] = 1
data.loc[(data['Emotion']=='분노'), 'Emotion'] = 2
data.loc[(data['Emotion']=='슬픔'), 'Emotion'] = 3
data.loc[(data['Emotion']=='중립'), 'Emotion'] = 4
data.loc[(data['Emotion']=='행복'), 'Emotion'] = 5
data.loc[(data['Emotion']=='혐오'), 'Emotion'] = 6
data

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:723: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


,Sentence,Emotion
0,언니 동생으로 부르는게 맞는 일인가요..??,0
1,그냥 내 느낌일뿐겠지?,0
2,아직너무초기라서 그런거죠?,0
3,유치원버스 사고 낫다던데,0
4,근데 원래이런거맞나요,0
...,...,...
38589,솔직히 예보 제대로 못하는 데 세금이라도 아끼게 그냥 폐지해라..,6
38590,재미가 없으니 망하지,6
38591,공장 도시락 비우생적임 아르바이트했는데 화장실가성 손도 않씯고 재료 담고 바닥 떨어...,6
38592,코딱지 만한 나라에서 지들끼리 피터지게 싸우는 센징 클래스 ㅉㅉㅉ,6


In [54]:
data['Emotion'].unique()
data['Emotion'].value_counts()

5    6037
1    5898
2    5665
0    5468
6    5429
3    5267
4    4830
Name: Emotion, dtype: int64

In [55]:
data.isnull().sum()

Sentence    0
Emotion     0
dtype: int64

In [56]:
data_list = []
for ques, label in zip(data['Sentence'], data['Emotion']):
  test_data = []
  test_data.append(ques)
  test_data.append(str(label))

  data_list.append(test_data)
data_list
# len(data_list)

[['언니 동생으로 부르는게 맞는 일인가요..??', '0'],
 ['그냥 내 느낌일뿐겠지?', '0'],
 ['아직너무초기라서 그런거죠?', '0'],
 ['유치원버스 사고 낫다던데', '0'],
 ['근데 원래이런거맞나요', '0'],
 [' 남자친구가 떠날까봐요', '0'],
 ['이거 했는데 허리가 아플수도 있나요? ;;', '0'],
 ['내가불안해서꾸는걸까..', '0'],
 [' 일주일도 안 남았당...ㅠㅠ', '0'],
 ['약은 최대한 안먹으려고 하는데좋은 음시있나요?0', '0'],
 [' 구직활동 하면서 남는시간은 뭘로 활용해야 되지..', '0'],
 ['괜찮은분 같아서 괜히 조급해지네요 ㅜ', '0'],
 [' 제가 스타일을 바꾸면 더 어색하게만 변할것같아서 ㅠ0', '0'],
 ['아내 있으면 여자 있는 술집가면 법 적으로 문제있나요?', '0'],
 [' 한소리들을지.... 도와주세요', '0'],
 ['도와주실분 없나요..', '0'],
 ['그냥 잡다한거말고학업에 관련된거 듣는게 더 좋은가요?', '0'],
 [' 가격은 반값....이거 질이 다른건가요??', '0'],
 ['마지막에 말씀 드렸던대로 글은 이제 내릴게요의도치않게 실시간베스트글에 올라가서 남친이 볼거 같아서요 ㅠㅠ', '0'],
 ['화장품회사다보니, 젊은 여자들이 많은거 같은데 걱정이네요..', '0'],
 [' 몸무게 1키로찌는건 아니겠죠?', '0'],
 ['o자 다리 많이 심한가요?', '0'],
 [' 안하는게 나을까요??', '0'],
 ['그정도로 더러운가요.........', '0'],
 [' 보통 가진통도 이렇게 오래가나요?', '0'],
 ['여자가 술취해서 먼저 전화하는거 짜증나요???', '0'],
 ['걱정이 큽니다', '0'],
 ['이거 무슨 증상이죠??ㅜㅜ', '0'],
 [' 다름 아니라 제동생 걱정 입니다', '0'],
 ['아무래도 무리겠죠?', '0'],
 ['애인으로써 정말 걱정됩니다', '0'],
 ['이러다 일주일전에 나오는

In [57]:
len(data_list)
data_list[38593]
data_list[30000]

['  요리하는것이 숙제하는것처럼 힘든저에게 용기나게 해주시고 할수 있을것같은 희망을 주셔서감사합니다!!', '5']

In [58]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer,vocab, max_len,
                 pad, pair):
   
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len,vocab=vocab, pad=pad, pair=pair)
        
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))
         

    def __len__(self):
        return (len(self.labels))

In [59]:
# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 10
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [60]:
#train & test 데이터로 나누기
from sklearn.model_selection import train_test_split

dataset_train, dataset_test = train_test_split(data_list, test_size=0.2, shuffle=True, random_state=34)
len(dataset_train)
# len(dataset_test)

30875

In [61]:
tok=tokenizer.tokenize
data_train = BERTDataset(dataset_train, 0, 1, tok, vocab, max_len, True, False)
data_test = BERTDataset(dataset_test,0, 1, tok, vocab,  max_len, True, False)

In [62]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [63]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=7,   ## 분류하고 싶은 감정 수 (긍정, 부정으로 할거면 2로 수정) ##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device),return_dict=False)
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out) # 64 6 

In [64]:
#BERT 모델 불러오기
# dr_rate 를 높여도 정확도가 높아지진 않음. 
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)
 
#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss() # 다중분류를 위한 대표적인 loss func

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

In [ ]:
train_history=[]
test_history=[]
loss_history=[]
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
         
        print(label.shape,out.shape)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
            train_history.append(train_acc / (batch_id+1))
            loss_history.append(loss.data.cpu().numpy())
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    #train_history.append(train_acc / (batch_id+1))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    test_history.append(test_acc / (batch_id+1))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/483 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


torch.Size([64]) torch.Size([64, 7])
epoch 1 batch id 1 loss 2.0293009281158447 train acc 0.09375
torch.Size([64]) torch.Size([64, 7])
torch.Size([64]) torch.Size([64, 7])
torch.Size([64]) torch.Size([64, 7])
torch.Size([64]) torch.Size([64, 7])
torch.Size([64]) torch.Size([64, 7])
torch.Size([64]) torch.Size([64, 7])
torch.Size([64]) torch.Size([64, 7])
torch.Size([64]) torch.Size([64, 7])
torch.Size([64]) torch.Size([64, 7])
torch.Size([64]) torch.Size([64, 7])
torch.Size([64]) torch.Size([64, 7])
torch.Size([64]) torch.Size([64, 7])
torch.Size([64]) torch.Size([64, 7])
torch.Size([64]) torch.Size([64, 7])
torch.Size([64]) torch.Size([64, 7])
torch.Size([64]) torch.Size([64, 7])
torch.Size([64]) torch.Size([64, 7])
torch.Size([64]) torch.Size([64, 7])
torch.Size([64]) torch.Size([64, 7])
torch.Size([64]) torch.Size([64, 7])
torch.Size([64]) torch.Size([64, 7])
torch.Size([64]) torch.Size([64, 7])
torch.Size([64]) torch.Size([64, 7])
torch.Size([64]) torch.Size([64, 7])
torch.Size([64

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

torch.Size([64]) torch.Size([64, 7])
epoch 7 batch id 1 loss 0.31964176893234253 train acc 0.890625
torch.Size([64]) torch.Size([64, 7])
torch.Size([64]) torch.Size([64, 7])
torch.Size([64]) torch.Size([64, 7])
torch.Size([64]) torch.Size([64, 7])
torch.Size([64]) torch.Size([64, 7])
torch.Size([64]) torch.Size([64, 7])
torch.Size([64]) torch.Size([64, 7])
torch.Size([64]) torch.Size([64, 7])
torch.Size([64]) torch.Size([64, 7])
torch.Size([64]) torch.Size([64, 7])
torch.Size([64]) torch.Size([64, 7])
torch.Size([64]) torch.Size([64, 7])
torch.Size([64]) torch.Size([64, 7])
torch.Size([64]) torch.Size([64, 7])
torch.Size([64]) torch.Size([64, 7])
torch.Size([64]) torch.Size([64, 7])
torch.Size([64]) torch.Size([64, 7])
torch.Size([64]) torch.Size([64, 7])
torch.Size([64]) torch.Size([64, 7])
torch.Size([64]) torch.Size([64, 7])
torch.Size([64]) torch.Size([64, 7])
torch.Size([64]) torch.Size([64, 7])
torch.Size([64]) torch.Size([64, 7])
torch.Size([64]) torch.Size([64, 7])
torch.Size([

  0%|          | 0/121 [00:00<?, ?it/s]

epoch 7 test acc 0.5263860192837466


  0%|          | 0/483 [00:00<?, ?it/s]

torch.Size([64]) torch.Size([64, 7])
epoch 8 batch id 1 loss 0.2270144373178482 train acc 0.921875
torch.Size([64]) torch.Size([64, 7])
torch.Size([64]) torch.Size([64, 7])
torch.Size([64]) torch.Size([64, 7])
torch.Size([64]) torch.Size([64, 7])
torch.Size([64]) torch.Size([64, 7])
torch.Size([64]) torch.Size([64, 7])
torch.Size([64]) torch.Size([64, 7])
torch.Size([64]) torch.Size([64, 7])
torch.Size([64]) torch.Size([64, 7])
torch.Size([64]) torch.Size([64, 7])
torch.Size([64]) torch.Size([64, 7])
torch.Size([64]) torch.Size([64, 7])
torch.Size([64]) torch.Size([64, 7])
torch.Size([64]) torch.Size([64, 7])
torch.Size([64]) torch.Size([64, 7])
torch.Size([64]) torch.Size([64, 7])
torch.Size([64]) torch.Size([64, 7])
torch.Size([64]) torch.Size([64, 7])


In [ ]:
def predict(predict_sentence):
  ## 재귀함수로 list 를 for 문으로 돌려서 함수 계속 호출

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, vocab, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        # print(out)
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            # print("2222") # 4 # 638
            # print(np.argmax(logits))
            
            if np.argmax(logits) == 0:
                test_eval.append("공포가")
            elif np.argmax(logits) == 1:
                test_eval.append("놀람이")
            elif np.argmax(logits) == 2:
                test_eval.append("분노가")
            elif np.argmax(logits) == 3:
                test_eval.append("슬픔이")
            elif np.argmax(logits) == 4:
                test_eval.append("중립이")
            elif np.argmax(logits) == 5:
                test_eval.append("행복이")
            elif np.argmax(logits) == 6:
                test_eval.append("혐오가")

        print(">> 입력하신 내용에서 " + test_eval[0] + " 느껴집니다.")

In [ ]:
#질문 무한반복하기! 0 입력시 종료
end = 1
while end == 1 :
    sentence = input("하고싶은 말을 입력해주세요 : ")
    if sentence == "0" :
        break
    predict(sentence)
    # print(out.shape)
    print("\n")